In [3]:
import os
import config as c
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.datasets import fetch_california_housing
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
from sklearn.model_selection import train_test_split
import plotly.graph_objs as go

from visualization.utils import plot_correlation_matrix, plot_scatterplot_matrix
from utils.metrics import NIC
from utils.dataset import *
from utils.metrics import NIC
from sklearn.metrics import mean_squared_error
from utils.model import Model, get_combine_predictions

init_notebook_mode(connected=True)

warnings.filterwarnings('ignore')
plt.rcParams ['figure.figsize'] = (13,8)
sns.set()
%config InlineBackend.figure_format = 'retina'

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
df_funnel, df_appl, df_aum, df_balance, df_client, df_com, df_deals, df_payments = get_dataset(True, False)

In [6]:
df_funnel

,client_id,sale_flg,sale_amount,contacts,feature_1,client_segment,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,region_cd
0,7513301859607023584,0,NaN,1,7,13.0,571533.0,15717.0,0.0,0.0,0.0,571852.0,472605.0,10.4,12548.0,86.0
1,9157009756404187626,0,NaN,1,3,13.0,3642369.0,94787.0,0.0,0.0,84823.0,3642369.0,3314257.0,8.9,77210.0,2.0
2,-1893104556496814867,0,NaN,1,5,16.0,352826.0,5500.0,0.0,6822.0,0.0,265893.0,204534.0,8.9,5508.0,52.0
3,6886062013213911831,0,NaN,1,4,3.0,6070615.0,40580.0,0.0,30401.0,0.0,2005731.0,1825051.0,7.9,40583.0,86.0
4,-8156468515495593794,1,138018.05,1,7,14.0,3642369.0,97156.0,81488.0,0.0,160308.0,3642369.0,3314257.0,10.4,78108.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21493,4662551505651924284,0,NaN,2,3,3.0,6070615.0,97616.0,21921.0,85635.0,89802.0,4826139.0,4391391.0,7.9,97626.0,86.0
21494,-8968737688687691353,0,NaN,1,9,14.0,1821184.0,2428.0,3952.0,0.0,13938.0,92273.0,71641.0,19.9,2440.0,78.0
21495,-5799097497074119478,0,NaN,1,2,16.0,6070615.0,64991.0,0.0,13450.0,0.0,3212569.0,2923175.0,7.9,64998.0,69.0
21496,-2264802671063321355,1,43882.85,2,8,4.0,2282408.0,5832.0,16185.0,0.0,82182.0,230683.0,177449.0,17.9,5845.0,59.0


# Создание фич

In [7]:
df = df_funnel.copy()

In [8]:
x = df[df['sale_amount'].fillna(0) > 0].drop(columns=['client_id', 'sale_flg', 'sale_amount', 'contacts']).to_numpy()
y = df[df['sale_amount'].fillna(0) > 0]['sale_amount'].to_numpy().reshape(-1, 1)

model_sale_amount = Model('model_sale_amount')
model_sale_amount.fit(x, y)
print(mean_squared_error(y, model_sale_amount.predict(x)))


x = df.drop(columns=['client_id', 'sale_flg', 'sale_amount', 'contacts']).to_numpy()
y = df['sale_flg'].to_numpy().reshape(-1, 1)

model_sale_flg = Model('model_sale_flg')
model_sale_flg.fit(x, y)
print(mean_squared_error(y, model_sale_flg.predict(x)))

x = df.drop(columns=['client_id', 'sale_flg', 'sale_amount', 'contacts']).to_numpy()

target = get_combine_predictions(x, model_sale_flg, model_sale_amount)
print('NIC:', NIC(target, df['sale_amount'], df['contacts']))

10it [00:10,  1.02s/it]
100%|██████████| 10/10 [00:00<00:00, 238.08it/s]
0it [00:00, ?it/s]

Models score: -442.96576232813004
1081675644.2592583


10it [00:44,  4.45s/it]
100%|██████████| 10/10 [00:00<00:00, 188.68it/s]


Models score: -2486.8992438110954
0.10807560582898804
NIC: 4439.753560796354
